In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np # For saving data

In [2]:
mp_pose = mp.solutions.pose
pose_detector = mp_pose.Pose(static_image_mode=False, # Process video
                             model_complexity=1, # 0, 1, or 2 (higher is slower but more accurate)
                             enable_segmentation=False, # Not needed for just landmarks
                             min_detection_confidence=0.5,
                             min_tracking_confidence=0.5)

In [3]:
video_folder = 'videos'
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

In [4]:
landmark_folder = 'landmarks'

# Create landmark folder if it doesn't exist
if not os.path.exists(landmark_folder):
    os.makedirs(landmark_folder)

# List video files
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

print(f"Starting landmark extraction for videos in '{video_folder}'...")
print(f"Saving landmarks to '{landmark_folder}'...")

# --- Loop Through Videos ---
for video_file in video_files:

    # Define paths
    video_path = os.path.join(video_folder, video_file)
    video_id = os.path.splitext(video_file)[0]
    save_landmark_path = os.path.join(landmark_folder, f"{video_id}.npy")

    # --- Check if .npy file already exists ---
    if os.path.exists(save_landmark_path):
        print(f"Skipping {video_file} - .npy file already exists.")
        continue
    # --- End of check ---

    print(f"Processing {video_file}...")
    cap = cv2.VideoCapture(video_path)

    # --- ADD CHECK: See if video opened successfully ---
    if not cap.isOpened():
        print(f"Warning: Could not open video file {video_file}. Skipping (likely corrupted).")
        cap.release() # Ensure it's released even if never opened properly
        continue
    # --- END CHECK ---

    frame_landmarks_list = []
    processed_successfully = True # Flag to track if processing completed

    try:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                # This could be end of video OR a read error
                # Check if it's truly the end or just an error
                current_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
                total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
                if current_pos < total_frames - 1 : # If not near the end, likely an error
                     print(f"  Warning: Frame read error in {video_file} near frame {int(current_pos)}. Skipping rest of video.")
                     processed_successfully = False # Mark as potentially incomplete
                break # Exit loop on error or end of video

            # Convert the BGR image to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_rgb.flags.writeable = False

            # Process the image and detect the pose
            results = pose_detector.process(image_rgb)

            image_rgb.flags.writeable = True

            # Extract landmarks if pose is detected
            if results.pose_landmarks:
                landmarks_current_frame = []
                for landmark in results.pose_landmarks.landmark:
                    landmarks_current_frame.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
                frame_landmarks_list.append(landmarks_current_frame)
            else:
                frame_landmarks_list.append([0.0] * 33 * 4)

    except cv2.error as e:
        print(f"  Error processing frame in {video_file}: {e}. Skipping rest of video.")
        processed_successfully = False # Mark as failed due to OpenCV error
    except Exception as e:
        print(f"  An unexpected error occurred processing {video_file}: {e}. Skipping rest of video.")
        processed_successfully = False # Mark as failed due to other error
    finally:
        cap.release() # Always release the video capture

    # --- Save the extracted landmarks only if processed successfully and frames exist ---
    if processed_successfully and frame_landmarks_list:
        video_landmark_data = np.array(frame_landmarks_list)
        np.save(save_landmark_path, video_landmark_data)
        # print(f"Saved landmarks for {video_file}")
    elif not processed_successfully:
         print(f"Skipped saving .npy for {video_file} due to processing errors.")
    else: # No frames in list, but no error flag triggered (e.g., empty video?)
        print(f"Warning: No frames/landmarks processed for {video_file}. No .npy file saved.")


pose_detector.close()
print("\nLandmark extraction complete!")

Starting landmark extraction for videos in 'videos'...
Saving landmarks to 'landmarks'...
Skipping 01457.mp4 - .npy file already exists.
Skipping 01467.mp4 - .npy file already exists.
Skipping 01468.mp4 - .npy file already exists.
Skipping 01469.mp4 - .npy file already exists.
Skipping 05296.mp4 - .npy file already exists.
Skipping 05304.mp4 - .npy file already exists.
Skipping 05305.mp4 - .npy file already exists.
Skipping 05307.mp4 - .npy file already exists.
Skipping 06176.mp4 - .npy file already exists.
Skipping 08608.mp4 - .npy file already exists.
Skipping 08935.mp4 - .npy file already exists.
Skipping 08936.mp4 - .npy file already exists.
Skipping 08937.mp4 - .npy file already exists.
Skipping 08938.mp4 - .npy file already exists.
Skipping 08942.mp4 - .npy file already exists.
Skipping 08944.mp4 - .npy file already exists.
Skipping 08945.mp4 - .npy file already exists.
Skipping 08946.mp4 - .npy file already exists.
Skipping 08948.mp4 - .npy file already exists.
Skipping 10577.mp

C:\Users\hp\OneDrive\Desktop\major project 2\WLASL\mediapipe_env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing 38524.mp4...
Processing 38525.mp4...
Processing 38527.mp4...
Processing 38529.mp4...
Processing 38530.mp4...
Processing 38532.mp4...
Processing 38533.mp4...
Processing 38534.mp4...
Processing 38538.mp4...
Processing 38539.mp4...
Processing 38540.mp4...
Processing 38541.mp4...
Processing 41578.mp4...
Processing 41602.mp4...
Skipped saving .npy for 41602.mp4 due to processing errors.
Processing 41604.mp4...
Processing 41606.mp4...
Processing 43213.mp4...
Processing 43224.mp4...
Processing 46287.mp4...
Processing 46302.mp4...
Processing 49184.mp4...
Processing 49185.mp4...
Processing 49186.mp4...
Processing 49244.mp4...
Processing 49253.mp4...
Processing 52550.mp4...
Processing 52562.mp4...
Processing 52564.mp4...
Processing 53363.mp4...
Processing 53372.mp4...
Processing 53373.mp4...
Processing 55656.mp4...
Processing 55665.mp4...
Processing 55666.mp4...
Processing 56552.mp4...
Processing 56556.mp4...
Processing 56557.mp4...
Processing 56558.mp4...
Processing 56563.mp4...
Proc